In [2]:
#드라이브 마운트
#디렉토리 변경
import os
os.chdir('/content/drive/MyDrive/ML/Data')

In [3]:
#1.CSV데이터 읽어오기("iris.csv")
import pandas as pd
csv_data = pd.read_csv("iris.csv")
csv_data.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Name
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [4]:
#2.특성데이터와 정답데이터 구분
iris_data = csv_data[['SepalLength','SepalWidth','PetalLength','PetalWidth']]
iris_target = csv_data['Name']

In [5]:
#3.훈련/데스트 데이터 분리
from sklearn.model_selection import train_test_split
train_data, test_data, train_target, test_target = train_test_split(iris_data, iris_target, random_state=42)

##XGBoost 파라미터들

###General parameters(일반 설정들)

booster[default = 'gbtree']:부스트 종류
* gbtree : 트리 기반 모델
* gblinear : 선형 모델

silent[default = 0]
* 0: 동작 메세지 프린트 함
* 1: 동작 메세지 프린트 안함

### Booster parameters(모델 파라미터)
* n_estimators[default=100] : 트리 모델의 개수

* early_stopping_rounds : loss에 더이상 진전이 없으면 멈춤. n_estimators가 높을 때 주로 사용(과적합을 방지)

* learning_rate[default=0.1] : 학습 단계별로 이전 결과를 얼마나 반영할지 설정.
일반적으로 0.01 ~ 0.2 사이의 값을 쓰며 너무 작게 하면 시간이 더 오래 걸림.

* min_child_weight[default=1] :
child 에서 필요한 모든 관측치에 대한 가중치의 최소합. 이 값보다 샘플 수가 작으면 leaf node가 된다. 너무 크게 설정하면 Under-fitting 될 수 있다.

* max_depth[default=6] :
트리의 최대 깊이. 일반적으로 3 ~ 10의 값.

* gamma[default=0] : 트리에서 추가적으로 가지를 나눌지 말지 결정하는 최소 손실 감소 값.
값을 크게 설정할 수록 과적합이 감소하는 효과.

* subsample[default=1] : 각 트리마다 데이터 샘플링 비율. over-fitting방지
일반적으로 0.5 ~ 1을 사용.

* colsample_bytree[default=1] : 각 트리마다 feature 샘플링 비율. 일반적으로 0.5 ~ 1을 사용.

* reg_lambda[default=1] : L2 규제(Ridge)

* reg_alpha [default=0] : L1 규제(Lasso)

* scale_pos_weight[default = 1] : 데이터가 불균형 할때 사용
보통은 음성 데이터 수 / 양성 데이터 수 값으로 함.

###Learning Task parameters
1. objective[default=reg:linear]
* binary:logistic : 이진분류를 위한 로지스틱 회귀, 클래스가 아닌 예측 확률 반환
* multi:softmax : softmax를 사용한 다중 클래스 분류, 클래스 반환
* multi:softprob : softmax를 사용, 클래스에 대한 예상 확률 반환
2. eval_metric[목적함수에 따라 default가 다름 회귀-rmse/ 분류-error]
* rmse : Root Mean Square Error
* mae : Mean Absolute error
* logloss : Negative log-likelihood
* error : Binary classification error rate(임계값 0.5)
* merror : multiclass classification error rate
* mlogloss : multiclass logloss
auc : area under curve
3. seed[default=0]
시드값 고정(나중에 재현 할때 같은 값을 출력하기 위해서 사용)

In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_target = le.fit_transform(train_target)
test_target = le.transform(test_target)

In [9]:
#4.모델 훈련 및 정확도(XGBClassifier(eval_metric='mlogloss'))
from xgboost import XGBClassifier
model = XGBClassifier(eval_metric='mlogloss')
model.fit(train_data, train_target)
print(model.score(train_data, train_target))
print(model.score(test_data, test_target))

1.0
1.0


In [12]:
pred_csv = pd.read_csv("iris_test.csv")
pred_data = pred_csv[['SepalLength','SepalWidth','PetalLength','PetalWidth']]
pred_target = pred_csv['Name']
pred_target = le.transform(pred_target)
print(model.predict(pred_data))

[0 0 1 2 2 0 0 1 0 2]


In [13]:
print(pred_target)

[0 0 1 2 2 0 0 1 0 2]
